# Limpieza de los datos de la Base de datos `bookmaker.db`

Antes de empezar a desarrollar la limpieza de los datos, hemos seleccionado como Kernel el entorno que hemos creado previamente

In [2]:
%pip install numpy
%pip install matplotlib
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3

Ahora vamos a desglosar la base de datos en 6 tabblas, separadas unas de otras

In [4]:
# Conectar a la base de datos SQLite
conn = sqlite3.connect("bookmaker.db")

# Listar las tablas en la base de datos
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)

tables


,name
0,clientes
1,empresa
2,equipos
3,partidos
4,apuestas
5,cuotas


In [5]:
# Cargar la tabla 'clientes' en un DataFrame
df_clientes = pd.read_sql("SELECT * FROM clientes", conn)
df_clientes.head()

,id,nombre,apellido,email,activado
0,1,Lope,Perales,pascualperera@vila.es,1
1,2,Isaura,Canet,gmerino@rocamora.es,1
2,3,Olga,Chaves,amayaprats@llado.com,1
3,4,Celestino,Hoyos,chedelgado@moraleda-pintor.com,1
4,5,Miguel,Milla,inigoprat@gmail.com,1


In [6]:
# Cargar la tabla 'empresa' en un DataFrame
df_empresa = pd.read_sql("SELECT * FROM empresa", conn)
df_empresa.head()

,id,razon_social,email,activado


In [7]:
# Cargar la tabla 'equipos' en un DataFrame
df_equipos = pd.read_sql("SELECT * FROM equipos", conn)
df_equipos.head()

,id,nombre,escudo,pais,puntaje,activado
0,1,Ajax,ajax.png,Paises Bajos,14.0,1
1,2,Atalanta,atalanta.png,Italia,6.0,1
2,3,Atlético,atletico.png,España,11.0,1
3,4,Barcelona,barcelona.png,España,17.0,1
4,5,Bayern,bayern.png,Alemania,16.0,1


In [8]:
# Cargar la tabla 'partidos' en un DataFrame
df_partidos = pd.read_sql("SELECT * FROM partidos", conn)
df_partidos.head()

,id,fecha,equipo_local,equipo_visistante,finalizado,ganador
0,1,2021-12-15 00:51:23.181740,29,2,0,None
1,2,2021-12-15 00:51:23.181740,31,19,0,None
2,3,2021-12-15 00:51:23.181740,25,24,0,None
3,4,2021-12-15 00:51:23.181740,11,6,0,None
4,5,2021-12-15 00:51:23.181740,4,5,0,None


In [9]:
# Cargar la tabla 'apuestas' en un DataFrame
df_apuestas = pd.read_sql("SELECT * FROM apuestas", conn)
df_apuestas.head()

,id,fecha,monto,equipo_ganador_id,partido,cliente,ganacia
0,1,2021-12-15 01:08:30.211674,300.0,2.0,1,88,891.0
1,2,2021-12-15 01:08:30.211674,300.0,29.0,1,88,891.0
2,3,2021-12-15 01:35:34.965516,300.0,2.0,1,88,891.0
3,4,2021-12-15 01:35:34.965516,300.0,29.0,1,88,891.0
4,5,2021-12-15 01:35:34.965516,300.0,2.0,1,88,891.0


In [10]:
# Cargar la tabla 'cuotas' en un DataFrame
df_cuotas = pd.read_sql("SELECT * FROM cuotas", conn)
df_cuotas.head()

,id,local,empate,visitante,partido_id
0,1,1.40,2.97,5.80,1
1,2,5.73,5.08,3.86,2
2,3,5.45,5.99,5.34,3
3,4,1.95,5.08,4.78,4
4,5,4.44,5.99,4.61,5


Los valores de la columna `ganador`en la tabla `partidos` están vacios (llenos de None)

In [11]:
df_partidos["ganador"].unique()

array([None], dtype=object)

En la tabla `apuestas` tambien hay valores nulos en la columna `equipo_ganador_id`

In [12]:
df_apuestas["equipo_ganador_id"].unique()

array([ 2., 29., nan,  6.])

In [13]:
print("None"), df_apuestas["equipo_ganador_id"].isnull().sum()

None


(None, 7)

## Problemas encontrados en los datos

1. En la tabla `apuestas`, los valores nulos en la columna `equipo_ganador_id` indican `apuestas` que se han realizado pero para las cuales el resultado del partido correspondiente aún no se ha determinado o no se ha registrado correctamente.

2. Los valores de la columna `ganador` en la tabla partidos están vacíos (llenos de None), lo cual sugiere que los resultados de algunos partidos no se han registrado.

3. Dentro de la tabla `apuestas` podemos encontrar que todas las apuestas se hacen desde el mismo id de usuario.

4. Dentro de la tabla `apuestas` encontramos que las apuestas para un mismo partido las ganan ambos equipos en casos diferentes.

## Insomnia Api

Por lo visto , Insomnia es una aplicación de escritorio gratuita que permite a los desarrolladores probar APIs. Facilita el envío de solicitudes HTTP y la organización de diferentes entornos para pruebas de APIs. Con Insomnia, puedes crear peticiones para probar métodos HTTP como GET, POST, PUT, DELETE, entre otros, así como configurar headers, añadir parámetros a las URLs, y enviar cuerpos de solicitud en varios formatos (como JSON o form-data). Esto lo hace especialmente útil para desarrollar y probar APIs REST y GraphQL.

Dentro de los datos proporcionados en el repositorio tenemos el archivo Insomnia_bookmaker, el cual vamos a utilizar para intentar solventar el problema con la falta de datos.

In [14]:
# Importar la biblioteca necesaria para leer el archivo
import json

# Ruta al archivo subido
file_path = 'Insomnia_bookmaker'

# Leer y cargar el contenido del archivo JSON
with open(file_path, 'r') as file:
    data = json.load(file)

# Mostrar la estructura básica del archivo para entender su contenido
data.keys()


dict_keys(['_type', '__export_format', '__export_date', '__export_source', 'resources'])

El contenido del archivo indica que hay varias peticiones definidas, incluyendo información sobre URLs, métodos HTTP (como GET y POST), cuerpos de las peticiones, y headers. Por ejemplo, aquí hay algunos detalles extraídos:

In [17]:
%pip install requests
import requests

Note: you may need to restart the kernel to use updated packages.


### Ejemplo de petición GET

In [18]:
# Realizar una petición GET
response = requests.get('http://127.0.0.1:8500/cuota/100')

# Verificar que la petición fue exitosa
if response.status_code == 200:
    print(response.json())  # Asumiendo que la respuesta es JSON
else:
    print(f'Error: {response.status_code}')


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8500): Max retries exceeded with url: /cuota/100 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002C0A6F2E5D0>: Failed to establish a new connection: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión'))

### Ejemplo de petición POST

In [19]:
headers = {'Content-Type': 'application/json'}
data = '{"partido_id": 1}'

# Realizar una petición POST
response = requests.post('http://127.0.0.1:8500/cuotas', headers=headers, data=data)

# Verificar que la petición fue exitosa
if response.status_code == 200:
    print(response.json())  # Asumiendo que la respuesta es JSON
else:
    print(f'Error: {response.status_code}')


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8500): Max retries exceeded with url: /cuotas (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002C0A6D85790>: Failed to establish a new connection: [WinError 10061] No se puede establecer una conexión ya que el equipo de destino denegó expresamente dicha conexión'))

## Acceso denegado

Como no tenemos permitido el acceso a esta API, intentaremos solucionarlo por otros métodos

In [20]:
df_apuestas['estado'] = df_apuestas['equipo_ganador_id'].apply(lambda x: 'pendiente' if pd.isnull(x) else 'definido')

In [21]:
df_apuestas

,id,fecha,monto,equipo_ganador_id,partido,cliente,ganacia,estado
0,1,2021-12-15 01:08:30.211674,300.0,2.0,1,88,891.0,definido
1,2,2021-12-15 01:08:30.211674,300.0,29.0,1,88,891.0,definido
2,3,2021-12-15 01:35:34.965516,300.0,2.0,1,88,891.0,definido
3,4,2021-12-15 01:35:34.965516,300.0,29.0,1,88,891.0,definido
4,5,2021-12-15 01:35:34.965516,300.0,2.0,1,88,891.0,definido
5,6,2021-12-15 01:35:34.965516,300.0,2.0,1,88,891.0,definido
6,7,2021-12-15 01:35:34.965516,300.0,29.0,1,88,891.0,definido
7,8,2021-12-15 01:35:34.965516,300.0,NaN,1,88,891.0,pendiente
8,9,2021-12-15 01:37:11.843995,300.0,NaN,1,88,891.0,pendiente
9,10,2021-12-15 01:38:57.727163,300.0,NaN,1,88,891.0,pendiente


In [23]:
df_apuestas.columns

Index(['id', 'fecha', 'monto', 'equipo_ganador_id', 'partido', 'cliente',
       'ganacia', 'estado'],
      dtype='object')

In [ ]:
columnas_eliminar = []

In [ ]:
df_apuestas.drop(columns=['estado'], inplace=True)

## Conclusiones

Después de un análisis exhaustivo de los datos, he llegado a la conclusión de que no tiene sentido hacer un análisis de esta base de datos con fines de hacer predicciones o poder conocer futuros movimientos de los usuarios. Esta conclusion se debe a que hay mucha incoherencia en los datos.

La limpieza de los datos supone practicamente cambiar por completo la tabla `apuestas` y hacer cambios tambien en partes de otras tablas como `partidos`.

El problema está en la incoherencia de los datos proporcionados dentro de la tabla `apuestas`, principalmente porque:
- Todas las apuuestas (excepto una) son realizadas por el usuario con id 88, esto no es una razon de mucho peso pero es extraño.
- Todas las apuestas (excepto una) son realizadas al mismo partido, esto no es una razon de mucho peso pero es extraño.
- Aún sabiendo que mayoritariamente las apuestas fueron realizadas hacia el partido cuyo id es el 1 (cuyo partido es el Villarreal vs Atalanta de 2021 y cuyo resultado fue de empate 2 a 2), los resultados __para el mismo partido__ fueron diferentes. Es decir, que para el mismo partido algunas apuestas dan como equipo ganador al Villarreal, otras al Atalanta, y otras se quedan vacias (que interpreté como un empate), lo cual no tiene ningún sentido.

Debido a todos estos problemas expuestos, llego a la conclusion de que no es rentable hacer un análisis de predicción sobre estos datos, porque obtendremos resutados que no van a estar basados en unos datos coherentes.

## Solución

La única solucion es, o bien cambiar los datos manualmente registro por registro, o bien encontrar una nueva tabla apuestas con datos correctos.